In [45]:
import h5pyd
import os
import numpy as np
import datetime
import julian

    Credentials on Matias's VM

In [27]:
api_user = 'terra'
api_password = '4rr3T'
api_endpoint = "http://terra-condo01.ncsa.illinois.edu:8000"
api_file = "terra.ncsa.illinois.edu"
api_path = f"basicfusion.{api_file}"

In [28]:
def getProxy(bf_file):
    year = bf_file.split('_')[4][:4]
    month = bf_file.split('_')[4][4:6]
    bf_path = f"{bf_file}.{year}%2E{month}.{api_path}"
    print(bf_path)
    h5f = h5pyd.File(bf_path, 'r', endpoint=api_endpoint, username=api_user, password=api_password)
    return h5f

In [37]:
f = getProxy('TERRA_BF_L1B_O66274_20120603081315_F000_V001')

TERRA_BF_L1B_O66274_20120603081315_F000_V001.2012%2E06.basicfusion.terra.ncsa.illinois.edu


In [43]:
SPATIAL_RESOLUTION = 0.5

NUM_POINTS = 1 / SPATIAL_RESOLUTION
NUM_LATS = int(180 / SPATIAL_RESOLUTION)
NUM_LONS = int(360 / SPATIAL_RESOLUTION)

# 
orbit_sw_sum  = np.zeros((NUM_LATS, NUM_LONS))
orbit_sw_num  = np.zeros((NUM_LATS, NUM_LONS), dtype='int16')
orbit_lw_sum  = np.zeros((NUM_LATS, NUM_LONS))

In [46]:
def get_descending(h5f, instrument):
    """
    Given a BF instance, return the starting and ending time of the descending node.
    Note that both times are stick to the MODIS 5-min granules.

    For CERES, return the julian time of the first/last descending MODIS granule in the bf file;
    For MODIS, return the first/last descending MODIS granule in the bf file;
    For MISR, return the first/last block of the descending MODIS granule in the bf file.
    
    Args:
        h5f (hdf5 instance): instance of a basic fusion file
        instrument (str)   : instrument from 'CERES', 'MISR', and 'MODIS'
    
    Returns:
        out_array (array): return MODIS granules, CERES start/end julian times, 
                                     or MISR start/end blocks depending on the instrument
    """

    MODIS_granules = [item[0] for item in h5f['MODIS'].items()]
    if len(MODIS_granules) == 0:
        print(">> IOError( no available MODIS granule in orbit {} )".format(bf_file))
        out_array = np.array([0, 0])

    else:
        descending_granules = []
        for igranule in MODIS_granules:     
            try:
                lats = h5f['MODIS/{}/_1KM/Geolocation/Latitude'.format(igranule)][:]
                lons = h5f['MODIS/{}/_1KM/Geolocation/Longitude'.format(igranule)][:]
            except KeyError:
                print(">> KeyError( cannot access lat/lon in {} )".format(igranule))
                continue

            # Process descending granules only (neutral granules are omitted)
            # May be improved by using a better criteria (?)
            cnt = 0
            for i in range(1, len(lats)):
                if all(lats[i]<lats[i-1]) == False:
                    cnt += 1
            if cnt >= 1000:
                # print(">> CriteriaError( this is not a descending granule {} )".format(igranule))
                continue
            else:
                descending_granules.append(igranule)

        descending_julian_bound = np.array([granuletime_to_jd(descending_granules[0]), granuletime_to_jd(descending_granules[-1], offset_mins=5)])

        if instrument.startswith('MODIS'):
            out_array = descending_granules
        elif instrument.startswith('CERES'):
            out_array = descending_julian_bound
        elif instrument.startswith('MISR'):
            # get MISR camera
            icam = instrument.split('.')[1]
            # get MISR BlockCenterTime
            try:
                bct = h5f['MISR/AN/BlockCenterTime'][:]
            except:
                print(">> IOError( cannot access MISR data )")
                out_array = np.array([0, 0])
            else:
                misr_block_julian = []
                for ibct in bct: 
                    yr, mon, day = str(ibct).split('-') # ibct: "b'2012-06-03T07:34:23.000532Z"
                    yr = int(yr[2:])                    # 2012
                    if yr == 0:
                        misr_block_julian.append(0)
                    else:
                        hr, mn, sec_decimal = day[3:].split(':') # "07:34:23.000532Z'"
                        sec = int(float(sec_decimal[:-2]))       # 23.000532
                        millisec = int(1000*(float(sec_decimal[:-2]) - sec)) # .000532 * 1000

                        dt = datetime.datetime(yr, int(mon), int(day[:2]), int(hr), int(mn), sec, millisec)
                        misr_block_julian.append(julian.to_jd(dt, fmt='jd'))
                
                misr_block_julian = np.array(misr_block_julian)
                misr_descending_blocks = np.where((misr_block_julian>=descending_julian_bound[0])&(misr_block_julian<=descending_julian_bound[1]))[0]

                out_array = misr_descending_blocks
    return out_array


def granuletime_to_jd(mod_granule_string, offset_mins=0):
    """
    Convert a MODIS granule string (e.g., 'granule_2012155_0700') to julian date (e.g., 2456081.7916666665)
    
    Args:
        mod_granule_string (str): MODIS granule string
        offset_mins (int, optional): offset minutes

    Returns:
        jd (int): julian date
    """
    yr = int(mod_granule_string.split('_')[1][:4])
    doy = int(mod_granule_string.split('_')[1][-3:])
    hr = int(mod_granule_string.split('_')[2][:2])
    mn = int(mod_granule_string.split('_')[2][-2:])

    dt = datetime.datetime(yr, 1, 1, hr, mn) + datetime.timedelta(days=doy-1, minutes=offset_mins)
    jd = julian.to_jd(dt, fmt='jd')    
    return jd

In [50]:
get_descending(f, 'MISR.AN')

array([ 27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138])